In [176]:
%load_ext autoreload
%autoreload 2
import sklearn.preprocessing as pre, scipy, numpy as np, matplotlib.pyplot as plt, glob, pickle, pyemma as py, sys, os
import mdtraj as md
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]= "2"
%matplotlib inline
from temp_tf_load import *
sys.path.append('../')
from hde import HDE, analysis
lag_time = 10
from pyemma.util.contexts import settings
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [177]:
# load pairwise distances and dihedral angles computed from simulation trajectory
#traj = md.load('/Users/aawad.19/Ferguson Lab/Documents/LSS_all_files/DESRES-20200729T124938Z-001/DESRES/DESRES-Trajectory_CLN025-0-protein/DESRES-Trajectory_CLN025-0-protein/CLN025-0-protein/CLN025-0-protein.pdb')
#top = md.load('/Users/aawad.19/Ferguson Lab/Documents/LSS_all_files/DESRES-20200729T124938Z-001/DESRES/DESRES-Trajectory_CLN025-0-protein/DESRES-Trajectory_CLN025-0-protein/CLN025-0-protein/CLN025-0-protein.pdb').topology

#traj_file = '/Users/aawad.19/Documents/Ferguson Lab/LSS_all_files/DESRES-20200729T124938Z-001/DESRES/DESRES-Trajectory_CLN025-0-protein/DESRES-Trajectory_CLN025-0-protein/CLN025-0-protein/CLN025-0-protein-000.dcd'
#traj_obj = md.load(traj_file, top='/Users/aawad.19/Documents/Ferguson Lab/LSS_all_files/DESRES-20200729T124938Z-001/DESRES/DESRES-Trajectory_CLN025-0-protein/DESRES-Trajectory_CLN025-0-protein/CLN025-0-protein/CLN025-0-protein.pdb')

trajs_files = ['/Users/aawad.19/Documents/Ferguson Lab/LSS_all_files/DESRES-20200729T124938Z-001/DESRES/DESRES-Trajectory_CLN025-0-protein/DESRES-Trajectory_CLN025-0-protein/CLN025-0-protein/CLN025-0-protein-%03d.dcd' % item
              for item in range(5)]
trajs_all = [md.load(item, top='/Users/aawad.19/Documents/Ferguson Lab/LSS_all_files/DESRES-20200729T124938Z-001/DESRES/DESRES-Trajectory_CLN025-0-protein/DESRES-Trajectory_CLN025-0-protein/CLN025-0-protein/CLN025-0-protein.pdb')
             for item in trajs_files]
traj_obj = md.join(trajs_all)

atoms = traj_obj.topology.select_atom_indices('heavy')
atom_pairs = traj_obj.topology.select_pairs(atoms,atoms)

pairwise_dis = md.compute_distances(traj_obj, atom_pairs, periodic=False, opt=True)

#atoms = top.select('backbone')
dihedrals = md.compute_dihedrals(traj_obj, ([0,1,2,3], [1,2,3,23], [2,24,25,26], [24,25,26,27], [23,24,25,44], [26,25,44,45], [25,44,45,46], [44,45,46,47], [44,45,46,56],[47,46,56,57],[46,56,57,58], [56,57,58,59], [57,58,70,71], [59,58,70,71], [58,70,71,72], [70,71,72,73], [70,71,72,85], [73,72,85,86], [72,85,86,87], [85,86,87,88], [86,87,99,100], [88,87,99,100], [87,99,100,101], [99,100,101,102], [99,100,101,106], [102,101,106,107], [101,106,107,108], [106,107,108,109], [107,108,120,121], [109,108,120,121], [109,108,120,121], [108,120,121,122], [120,121,122,123], [120,122,123,144], [121,122,144,145], [122,144,145,146], [144,145,146,155]), periodic=False, opt=True)

scaler = pre.MinMaxScaler(feature_range=(-1, 1))
pairwise_dis_s = scaler.fit_transform(pairwise_dis)
cossin_dihedrals = np.concatenate([np.cos(dihedrals), np.sin(dihedrals)], axis=1)

features = np.concatenate([pairwise_dis_s, cossin_dihedrals], axis=1)


## kernel tICA

In [178]:
from temp_ktica import *

In [180]:
# find landmarks, this may take a while.  We use 5000 landmarks in our paper, but it 
# takes very long time to do clustering, so here we use 2000 instead
from sklearn.cluster import KMeans
n_landmarks = 1200

kmeans = KMeans(init='k-means++', n_clusters=n_landmarks, n_init=10)
kmeans.fit(features[::8])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=1200, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [ ]:
#print(features[::9].shape)

In [190]:
# take longer than previous cell
temp_dataset = features
temp_landmarks = kmeans.cluster_centers_
for sigma in [25.0]:    
    ktica = Kernel_tica(3, lag_time=10, gamma=1./(2 * sigma ** 2), n_components_nystroem=len(temp_landmarks), 
                       landmarks = temp_landmarks)
    ktica_coords = ktica.fit_transform([temp_dataset])[0]
    print(sigma, ktica._tica.timescales[:5])
    ktica_timescales = ktica._tica.timescales[:3]

using landmarks
25.0 [1471.91375689  117.90001077   80.57837186   58.3444239    44.69377576]


In [6]:
# these are precomputed ktica coordinates and timescales
# ktica_coords = np.load('/home/kengyangyao/data/hde_paper_results/adp_ktica_coords.npy')
# ktica_timescales = np.load('/home/kengyangyao/data/hde_paper_results/adp_ktica_timescales.npy')

## HDE

In [182]:
skip = 1
from keras.callbacks import EarlyStopping
earlyStopping = EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='min', restore_best_weights=True)
hde = HDE(4352, n_components=3, lag_time=lag_time // skip, dropout_rate=0, batch_size=50000, n_epochs=200, 
          validation_split=0.2, batch_normalization=True, learning_rate = 0.01,
          callbacks=[earlyStopping])
#45
#batch_size = 50000

In [183]:
hde.r_degree = 10    # pre-training, to speed up discovery of first few slowest modes
hde.batch_size = 100000
#200000
hde.n_epochs = 50
hde.fit(features)
hde._recompile = True
hde.r_degree = 5
hde.fit(features)
hde.batch_size = 250000
#500000
hde._recompile = True
hde.r_degree = 2
hde.n_epochs = 100
hde.fit(features)

Epoch 1/50
49990/49990 [==============================] - 10s 206us/step - loss: -1.5266
Epoch 2/50
49990/49990 [==============================] - 12s 231us/step - loss: -1.6405
Epoch 3/50
49990/49990 [==============================] - 13s 253us/step - loss: -1.9619
Epoch 4/50
49990/49990 [==============================] - 13s 268us/step - loss: -2.1498
Epoch 5/50
49990/49990 [==============================] - 15s 298us/step - loss: -2.3000
Epoch 6/50
49990/49990 [==============================] - 12s 233us/step - loss: -2.3948
Epoch 7/50
49990/49990 [==============================] - 11s 217us/step - loss: -2.4639
Epoch 8/50
49990/49990 [==============================] - 19s 384us/step - loss: -2.5213
Epoch 9/50
49990/49990 [==============================] - 10s 197us/step - loss: -2.5655
Epoch 10/50
49990/49990 [==============================] - 11s 211us/step - loss: -2.5889
Epoch 11/50
49990/49990 [==============================] - 14s 285us/step - loss: -2.7051
Epoch 12/50
49990/4

Epoch 43/50
49990/49990 [==============================] - 14s 273us/step - loss: -3.6229
Epoch 44/50
49990/49990 [==============================] - 8s 164us/step - loss: -3.6272
Epoch 45/50
49990/49990 [==============================] - 10s 197us/step - loss: -3.6333
Epoch 46/50
49990/49990 [==============================] - 15s 297us/step - loss: -3.6369
Epoch 47/50
49990/49990 [==============================] - 12s 238us/step - loss: -3.6388
Epoch 48/50
49990/49990 [==============================] - 7s 150us/step - loss: -3.6484
Epoch 49/50
49990/49990 [==============================] - 14s 289us/step - loss: -3.6486
Epoch 50/50
49990/49990 [==============================] - 11s 226us/step - loss: -3.6570
Epoch 1/100
49990/49990 [==============================] - 13s 258us/step - loss: -3.6584
Epoch 2/100
49990/49990 [==============================] - 17s 345us/step - loss: -3.2953
Epoch 3/100
49990/49990 [==============================] - 9s 189us/step - loss: -3.3671
Epoch 4/100
4

Epoch 84/100
49990/49990 [==============================] - 9s 173us/step - loss: -3.8161
Epoch 85/100
49990/49990 [==============================] - 8s 161us/step - loss: -3.8169
Epoch 86/100
49990/49990 [==============================] - 8s 153us/step - loss: -3.8177
Epoch 87/100
49990/49990 [==============================] - 10s 199us/step - loss: -3.8188
Epoch 88/100
49990/49990 [==============================] - 8s 155us/step - loss: -3.8188
Epoch 89/100
49990/49990 [==============================] - 17s 332us/step - loss: -3.8195
Epoch 90/100
49990/49990 [==============================] - 9s 171us/step - loss: -3.8186
Epoch 91/100
49990/49990 [==============================] - 10s 195us/step - loss: -3.8179
Epoch 92/100
49990/49990 [==============================] - 10s 209us/step - loss: -3.8199
Epoch 93/100
49990/49990 [==============================] - 9s 183us/step - loss: -3.8223
Epoch 94/100
49990/49990 [==============================] - 16s 328us/step - loss: -3.8244
Epoch

HDE(activation=None, batch_normalization=True, batch_size=250000,
    callbacks=[<keras.callbacks.callbacks.EarlyStopping object at 0x1a3a26e690>],
    dropout_rate=0, hidden_layer_depth=None, hidden_size=None, input_size=4352,
    l2_regularization=0.0, lag_time=10, latent_space_noise=0,
    learning_rate=0.01, n_components=3, n_epochs=100, reversible=True,
    validation_split=0.2, verbose=True)

In [165]:
hde.r_degree = 2
hde.n_epochs = 100
hde.fit(features)

Epoch 1/100
39990/39990 [==============================] - 11s 272us/step - loss: -3.6727
Epoch 2/100
39990/39990 [==============================] - 8s 194us/step - loss: -3.6753
Epoch 3/100
39990/39990 [==============================] - 6s 148us/step - loss: -3.6777
Epoch 4/100
39990/39990 [==============================] - 5s 117us/step - loss: -3.6790
Epoch 5/100
39990/39990 [==============================] - 5s 126us/step - loss: -3.6792
Epoch 6/100
39990/39990 [==============================] - 3s 72us/step - loss: -3.6804
Epoch 7/100
39990/39990 [==============================] - 6s 146us/step - loss: -3.6826
Epoch 8/100
39990/39990 [==============================] - 4s 103us/step - loss: -3.6836
Epoch 9/100
39990/39990 [==============================] - 5s 137us/step - loss: -3.6837
Epoch 10/100
39990/39990 [==============================] - 6s 151us/step - loss: -3.6849
Epoch 11/100
39990/39990 [==============================] - 6s 145us/step - loss: -3.6853
Epoch 12/100
39990/

Epoch 93/100
39990/39990 [==============================] - 6s 158us/step - loss: -3.7023
Epoch 94/100
39990/39990 [==============================] - 6s 154us/step - loss: -3.7025
Epoch 95/100
39990/39990 [==============================] - 6s 155us/step - loss: -3.7026
Epoch 96/100
39990/39990 [==============================] - 6s 158us/step - loss: -3.7026
Epoch 97/100
39990/39990 [==============================] - 4s 109us/step - loss: -3.7027
Epoch 98/100
39990/39990 [==============================] - 6s 144us/step - loss: -3.7030
Epoch 99/100
39990/39990 [==============================] - 3s 71us/step - loss: -3.7030
Epoch 100/100
39990/39990 [==============================] - 3s 65us/step - loss: -3.7032


HDE(activation=None, batch_normalization=True, batch_size=250000,
    callbacks=[<keras.callbacks.callbacks.EarlyStopping object at 0x1a2f4edcd0>],
    dropout_rate=0, hidden_layer_depth=None, hidden_size=None, input_size=4352,
    l2_regularization=0.0, lag_time=10, latent_space_noise=0,
    learning_rate=0.01, n_components=3, n_epochs=100, reversible=True,
    validation_split=0.2, verbose=True)

In [184]:
hde_coords = hde.transform(features)
hde_timescales = hde.timescales_
print(hde_timescales)
print(hde_coords)

#cluster = py.coordinates.cluster_kmeans(hde_coords, k=200, n_jobs=1, max_iter=50)
#print(cluster)

[2063.09231095  520.24998669  150.20381646]
[[ 1.2419244   0.21987318 -0.35559627]
 [-0.12071919  0.09691165  0.32312223]
 [ 1.1593312   0.13921878  0.19697236]
 ...
 [-0.82291275 -0.00863858 -0.02370763]
 [-0.82291263 -0.00863834 -0.0237062 ]
 [-0.82291263 -0.00863834 -0.0237062 ]]


In [185]:
# save hde_coords as a file
np.save('hde_coords_chig.npy', hde_coords, allow_pickle=True, fix_imports=True)

In [186]:
# hde save as .pkl
filename = 'hde_model2_chig.pkl'
pickle.dump(hde, open(filename, 'wb'))

In [187]:
# loading in hde as .pkl and testing
hde = pickle.load(open(filename, 'rb'))

hde_timescales = hde.timescales_
print(hde_timescales)

#with open('lss_files/hde_model_chig.pkl', 'rb') as f:
#    srv = pickle.load(f)

[2063.09231095  520.24998669  150.20381646]


## MSM

In [194]:
tica = py.coordinates.tica(features, lag=lag_time, var_cutoff=0.99)
tica_coords = np.concatenate(tica.get_output())

AttributeError: 'function' object has no attribute 'children'

In [192]:
print(tica_coords)
print(tica_coords.shape)

[[-1.434792    0.5292531   1.337214   ...  0.00209512 -0.03079049
  -0.49156755]
 [-1.2131928   0.8993494   0.6746466  ...  0.14062645 -0.28177953
  -0.19445282]
 [-1.3411576   1.408441    0.6377958  ...  0.01261535  0.22428791
  -0.4185481 ]
 ...
 [ 0.64765173 -0.218882   -0.07262878 ... -0.15228854  0.07980075
  -0.29207638]
 [ 0.5509988  -0.41100702 -0.06193169 ...  0.28497922 -0.06418439
   0.25507587]
 [ 0.5469359   0.25415653  0.09602655 ... -0.14683363  0.02261833
  -0.20693459]]
(10000, 2181)


In [171]:
with settings(show_progress_bars=False):
    cluster = py.coordinates.cluster_kmeans(tica_coords, k=150, n_jobs=1, max_iter=50)

In [172]:
with settings(show_progress_bars=False):
    msm = py.msm.bayesian_markov_model(cluster.dtrajs, lag=lag_time)

print("Fraction of states used = {:.2f}".format(msm.active_state_fraction))
print("Fraction of counts used = {:.2f}".format(msm.active_count_fraction))

Fraction of states used = 0.98
Fraction of counts used = 1.00


In [173]:
eigvec = msm.eigenvectors_right()
dtrajs = np.concatenate(cluster.dtrajs)
print(eigvec.shape)
print(dtrajs.shape)

msm_coords = eigvec[dtrajs,1:6]
msm_timescales = msm.sample_mean('timescales', k=20)
print(msm_timescales[:3])

msm_timescales = msm.timescales(k=20)
print(msm_timescales)

(147, 147)
(10000,)


IndexError: index 148 is out of bounds for axis 0 with size 147

## compare results

In [ ]:
plt.rcParams['font.size'] = 14
def get_adp_state_plot(n_models):    
    skip = 10
    fig, axes = plt.subplots(n_models, 3)
    fig.set_size_inches(10, 3 * n_models + 1)

    coords_list = [ktica_coords, hde_coords, msm_coords][:n_models]
    timescales_list = [ktica_timescales, hde_timescales, msm_timescales][:n_models]
    for index_row in range(n_models):
        for item in range(3):
            temp_coords = coords_list[index_row][::skip, item]
            temp_coords *= np.sign(np.sum(temp_coords))
            ax = axes[index_row, item]
            im = ax.scatter(dihedrals[::skip, 0], dihedrals[::skip, 1], s=0.5, 
                                    c=temp_coords, cmap=plt.cm.RdBu)
            fig.colorbar(im, ax=ax, label='$\\tilde{\psi}_%d\ (t_%d = %d$ ps)' % (
                item + 1, item+1, timescales_list[index_row][item] * 2), fraction=0.046, pad=0.04)
            ax.text(-2, 0, '$\\alpha$', color='white')
            ax.text(-2.5, 2.2, '$\\beta$', color='white')
            ax.text(-1.5, 2.2, '$P_{//}$', color='white')
            ax.text(1.5, 0, '$\\alpha_L$')
            ax.text(1.5, 2.5, '$\\gamma$')
            ax.axis('square')
            ax.set_xlim([-np.pi, np.pi])
            ax.set_ylim([-np.pi, np.pi])
            if index_row == n_models - 1: ax.set_xlabel('$\phi$')
            if item == 0: ax.set_ylabel('$\psi$')

    row_label_list = ['kTICA', 'HDE', 'MSM'][:n_models]
    pad = 30 
    for ax, row in zip(axes[:, 0], row_label_list):
        ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
                    xycoords=ax.yaxis.label, textcoords='offset points',
                    ha='right', va='center', fontsize=20)
    plt.subplots_adjust(wspace=0.7, hspace=0.05)
    return fig

fig = get_adp_state_plot(3)